In [ ]:
#link for dataset https://www.kaggle.com/rmisra/news-category-dataset
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"freak747","key":"fa94e6b03d732b35df0cabcf283d8dbc"}'}

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rmisra/news-category-dataset

 35% 9.00M/25.4M [00:00<00:00, 35.3MB/s]
100% 25.4M/25.4M [00:00<00:00, 64.3MB/s]


In [ ]:
!ls

kaggle.json  news-category-dataset.zip	sample_data


In [ ]:
!unzip 'news-category-dataset.zip'

Archive:  news-category-dataset.zip
  inflating: News_Category_Dataset_v2.json  


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
df1 = pd.read_json("/content/News_Category_Dataset_v2.json",lines=True)

In [ ]:
df1.head()

In [ ]:
len(df1.index)

200853

In [ ]:
df2 = df1["headline"]

In [ ]:
df2.head()

0    There Were 2 Mass Shootings In Texas Last Week...
1    Will Smith Joins Diplo And Nicky Jam For The 2...
2      Hugh Grant Marries For The First Time At Age 57
3    Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4    Julianna Margulies Uses Donald Trump Poop Bags...
Name: headline, dtype: object

In [ ]:
!pip install --upgrade gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

     |████████████████████████████████| 24.2MB 1.4MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result
stemmer = SnowballStemmer("english")

In [ ]:
stemmer = SnowballStemmer("english")
processed_docs = []

for doc in df2:
    processed_docs.append(preprocess(doc))

In [ ]:
print(len(processed_docs))
print(processed_docs[:2])

200853
[['mass', 'shoot', 'texa', 'week'], ['smith', 'join', 'diplo', 'nicki', 'world', 'offici', 'song']]


In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    count = count +1
    if count > 4 : break
    print(k, v)

0 mass
1 shoot
2 texa
3 week


In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 15
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 28 ("trump") appears 1 time.
Word 85 ("putin") appears 1 time.
Word 87 ("edward") appears 1 time.
Word 88 ("love") appears 1 time.
Word 89 ("snowden") appears 1 time.
Word 90 ("vladimir") appears 1 time.


In [ ]:
# Save the Dict and Corpus
dictionary.save('dictionary.dict')  # save dict to disk
gensim.corpora.MmCorpus.serialize('bow_corpus.mm', bow_corpus)  # save corpus to disk

In [ ]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 20, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

In [ ]:
from gensim import corpora, models, similarities
lda_model.save('lda_newscatergory')

In [ ]:
lda_model.show_topics()

In [ ]:
from gensim import corpora, models, similarities
lda_model = gensim.models.LdaModel.load('lda_newscatergory')

In [ ]:
test = str(input("Example text: "))

Example text: Most people have faced rejection at least once in their life: a breakup with a significant other, a rejection from a dream college, or a job opportunity slipping away from a dream organisation. Most of us are also no strangers to the heartbreak and negative thoughts the rejection brings with it.  Justin Barker, a higher education/student affairs specialist with a number of companies, talks about how his experience of rejection was both the greatest and the worst thing to have happened to him.


In [ ]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(test))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.21768394112586975	 Topic: 0.081*"divorc" + 0.052*"health" + 0.035*"idea" + 0.024*"model" + 0.023*"care"
Score: 0.1589646339416504	 Topic: 0.078*"home" + 0.070*"way" + 0.053*"photo" + 0.031*"star" + 0.019*"leav"
Score: 0.13530489802360535	 Topic: 0.059*"thing" + 0.049*"need" + 0.049*"know" + 0.033*"mother" + 0.024*"gift"
Score: 0.06811115145683289	 Topic: 0.077*"love" + 0.064*"kid" + 0.042*"talk" + 0.026*"spring" + 0.023*"stop"
Score: 0.06635695695877075	 Topic: 0.132*"photo" + 0.088*"best" + 0.053*"world" + 0.043*"dress" + 0.037*"celebr"
Score: 0.0652586966753006	 Topic: 0.070*"photo" + 0.064*"style" + 0.026*"secret" + 0.025*"open" + 0.024*"poll"
Score: 0.06385715305805206	 Topic: 0.054*"tip" + 0.049*"travel" + 0.040*"sleep" + 0.036*"famili" + 0.031*"holiday"
Score: 0.038426902145147324	 Topic: 0.054*"chang" + 0.032*"trump" + 0.025*"think" + 0.021*"propos" + 0.019*"plan"
Score: 0.038304347544908524	 Topic: 0.033*"reason" + 0.027*"happi" + 0.022*"state" + 0.021*"death" + 0.020*